In [1]:
import pulp
import pandas as pd
import chardet
import os
import requests
import time
import json

#### FFS 6 gameweek projections

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from dotenv import load_dotenv

In [3]:
load_dotenv()

ffs_usr = os.getenv("FFS_LOGIN")
ffs_pass = os.getenv("FFS_PASS")

In [8]:
def scrape_projections():
    chrome_options = Options()
    #chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    service = Service("/opt/homebrew/bin/chromedriver")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    login_url = "https://www.fantasyfootballscout.co.uk/wp-login.php"
    driver.get(login_url)

    # Wait until the username field is available, then fill in the credentials
    wait = WebDriverWait(driver, 10)
    username_field = wait.until(EC.presence_of_element_located((By.NAME, "log")))
    password_field = driver.find_element(By.NAME, "pwd")

    username_field.send_keys(ffs_usr)
    password_field.send_keys(ffs_pass)

    # Click the submit button (WordPress login form uses id 'wp-submit')
    submit_button = driver.find_element(By.ID, "wp-submit")
    submit_button.click()

    # Wait a few seconds for login to complete
    time.sleep(3)

    # --- Navigate to the Projections Page ---
    projections_url = "https://members.fantasyfootballscout.co.uk/projections/six-game-projections/"
    driver.get(projections_url)

    # Wait for the projections table to load by waiting for the element with class "stats"
    table_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.stats")))
    # Optionally wait a little extra to ensure all JS rendering is complete
    time.sleep(2)

    # --- Extract the Table ---
    # Get the page source after the table has loaded
    html = driver.page_source

    # Use pandas to read all tables from the HTML
    tables = pd.read_html(html)

    # Look for the table that has the expected columns (e.g., "Name", "Team", etc.)
    projections_df = None
    for table in tables:
        if "Name" in table.columns and "Team" in table.columns:
            projections_df = table
            break

    if projections_df is not None:
        print("Extracted projections table")
    else:
        print("Projections table not found.")

    # Close the browser session
    driver.quit()

    df = projections_df.copy()

    df = df.iloc[:, :-2]  # drops the last two columns

    for col in df.columns:
        if col.startswith("GW"):
            new_col = col.replace("GW", "") 
            df.rename(columns={col: new_col}, inplace=True)

    df.rename(
        columns={
            "Name": "name",
            "Team": "team",
            "Pos": "position",
            "FPL Price": "value"
        },
        inplace=True
    )

    # Melt the DataFrame so each GW column becomes a row
    df = df.melt(
        id_vars=["name", "team", "position", "value"],
        var_name="gameweek",
        value_name="xP"
    )

    df['gameweek'] = df['gameweek'].astype('int')
    df['value'] = df['value'].astype('float64')
    df['xP'] = df['xP'].astype('float64')

    return df

#### Current team

In [5]:
def fpl_team(team_id, gameweek):
    picks_url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gameweek}/picks/"
    picks_response = requests.get(picks_url)
    picks_data = picks_response.json()

    # Get player data from the bootstrap-static endpoint
    bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    bootstrap_response = requests.get(bootstrap_url)
    bootstrap_data = bootstrap_response.json()

    # Create a mapping from player id to player name
    player_dict = {player['id']: player['web_name'] for player in bootstrap_data['elements']}

    # Add the player name to each pick in your picks_data
    for pick in picks_data['picks']:
        player_id = pick['element']
        pick['player_name'] = player_dict.get(player_id, "Unknown")

    picks_df = pd.DataFrame(picks_data['picks'])

    return picks_df

#### Optimiser

In [ ]:
def solve_multi_period_fpl(budget, data, start_gameweek, end_gameweek, 
                           bench_boost_gameweek=0, free_hit_gameweek=0, 
                           wildcard_gameweek=0, base_team=None):

    # Validate bench_boost_gameweek: 0 means no bench boost.
    if bench_boost_gameweek != 0 and (bench_boost_gameweek < start_gameweek - 1 or bench_boost_gameweek > end_gameweek + 1):
        raise ValueError("bench_boost_gameweek must be 0 or between start_gameweek and end_gameweek")
    # Validate free_hit_gameweek: 0 means no free hit; free hit should not be in the first gameweek.
    if free_hit_gameweek != 0:
        if free_hit_gameweek < start_gameweek or free_hit_gameweek > end_gameweek:
            raise ValueError("free_hit_gameweek must be 0 or between start_gameweek and end_gameweek")
        if free_hit_gameweek == start_gameweek:
            raise ValueError("free_hit_gameweek cannot be the first gameweek")
    # Validate wildcard_gameweek: 0 means no wildcard.
    if wildcard_gameweek != 0 and (wildcard_gameweek < start_gameweek or wildcard_gameweek > end_gameweek):
        raise ValueError("wildcard_gameweek must be 0 or between start_gameweek and end_gameweek")
    # If base_team is provided and is a DataFrame, extract the player names.
    if base_team is not None:
        if isinstance(base_team, pd.DataFrame):
            base_team = list(base_team['player_name'].unique())
        if len(base_team) != 15:
            raise ValueError("The base_team must contain exactly 15 players.")

    
    # Filter data for the specified gameweeks.
    filtered_data = data[(data['gameweek'] >= start_gameweek) & 
                         (data['gameweek'] <= end_gameweek)]
    
    # Create a dictionary for quick access to player data.
    player_data = {(row['name'], row['gameweek']): row for _, row in filtered_data.iterrows()}
    
    # Create the optimization model.
    model = pulp.LpProblem("Multi_Period_FPL_Optimization", pulp.LpMaximize)

    # Sets.
    gameweeks = range(start_gameweek, end_gameweek + 1)
    players = filtered_data['name'].unique()
    positions = filtered_data['position'].unique()
    teams = filtered_data['team'].unique()

    # Decision Variables.
    squad = pulp.LpVariable.dicts("squad", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    captain = pulp.LpVariable.dicts("captain", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_in = pulp.LpVariable.dicts("transfer_in", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_out = pulp.LpVariable.dicts("transfer_out", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    free_transfers = pulp.LpVariable.dicts("free_transfers", gameweeks, lowBound=0, upBound=5, cat='Integer')
    paid_transfers = pulp.LpVariable.dicts("paid_transfers", gameweeks, lowBound=0, cat='Integer')

    # Objective:
    # In the bench boost week (if > 0) use the entire squad (15 players), otherwise just the starting lineup.
    # Transfer costs are applied in all weeks except the free hit week.
    model += pulp.lpSum(
        player_data.get((p, gw), {}).get('xP', 0) * (
            (squad[p, gw] if bench_boost_gameweek > 0 and gw == bench_boost_gameweek else lineup[p, gw])
            + captain[p, gw] + 0.1 * vicecap[p, gw]
        )
        for p in players for gw in gameweeks
    ) - pulp.lpSum(
        4 * paid_transfers[gw] for gw in gameweeks if gw != free_hit_gameweek
    )
    
    # Set starting free transfers.
    model += free_transfers[start_gameweek] == 0

    # Constraints for each gameweek.
    for gw in gameweeks:
        # If we have a base team and we are at the initial gameweek, fix the squad.
        if gw == start_gameweek and base_team is not None:
            for p in players:
                if p in base_team:
                    model += squad[p, gw] == 1
                else:
                    model += squad[p, gw] == 0

        # Squad size.
        model += pulp.lpSum(squad[p, gw] for p in players) == 15

        # Starting lineup size.
        model += pulp.lpSum(lineup[p, gw] for p in players) == 11

        # Exactly one captain and one vice-captain.
        model += pulp.lpSum(captain[p, gw] for p in players) == 1
        model += pulp.lpSum(vicecap[p, gw] for p in players) == 1

        # Lineup, captain, and vice-captain must be members of the squad.
        for p in players:
            model += lineup[p, gw] <= squad[p, gw]
            model += captain[p, gw] <= lineup[p, gw]
            model += vicecap[p, gw] <= lineup[p, gw]
            model += captain[p, gw] + vicecap[p, gw] <= 1

        # Position constraints.
        for pos in positions:
            # Ensure the correct number of players per position in the squad.
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('position') == pos) \
                     == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
            # For the starting lineup, enforce positional minimums/maximums.
            if pos == 'GK':
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) == 1
            elif pos in ['DEF', 'MID']:
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 3
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 5
            else:  # FWD.
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 1
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 3

        # Budget constraint.
        model += pulp.lpSum(player_data.get((p, gw), {}).get('value', 0) * squad[p, gw] for p in players) <= budget

        # Team limit (max 3 players from the same team).
        for team in teams:
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('team') == team) <= 3

        # Transfer constraints for gameweeks after the first.
        if gw > start_gameweek:
            # ----- Free Hit Week -----
            if free_hit_gameweek != 0 and gw == free_hit_gameweek:
                # In the free hit week, allow a completely new squad.
                model += free_transfers[gw] == free_transfers[gw-1]
                # (Transfers in/free hit week are not counted, so we do not add transfer in/out constraints.)
            
            # ----- Week After Free Hit: Reversion -----
            elif free_hit_gameweek != 0 and gw == free_hit_gameweek + 1:
                # Force the squad to revert to the team from before the free hit.
                for p in players:
                    model += squad[p, gw] == squad[p, free_hit_gameweek - 1]
                # Also reset free transfers as if no transfers had been made in the free hit week.
                model += free_transfers[gw] == free_transfers[free_hit_gameweek - 1]
            
            # ----- Wildcard Week -----
            elif wildcard_gameweek != 0 and gw == wildcard_gameweek:
                # In a wildcard gameweek, allow any number of transfers by not linking the squad
                # to the previous gameweek and by bypassing the transfer-in/out constraints.
                model += free_transfers[gw] == 1  # Reset free transfers for subsequent weeks.
                # No continuity constraint is enforced here.
            
            # ----- Normal Weeks -----
            else:
                model += free_transfers[gw] == free_transfers[gw-1] + 1 - pulp.lpSum(transfer_in[p, gw-1] for p in players)
                model += free_transfers[gw] <= 5 
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == pulp.lpSum(transfer_out[p, gw] for p in players)
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == free_transfers[gw] + paid_transfers[gw]
                for p in players:
                    model += squad[p, gw] == squad[p, gw-1] + transfer_in[p, gw] - transfer_out[p, gw]
                    model += transfer_in[p, gw] + transfer_out[p, gw] <= 1

    # Solve the model.
    model.solve()

    # Process and return results.
    if pulp.LpStatus[model.status] == 'Optimal':
        results = []
        for gw in gameweeks:
            picks = []
            transfers_in = []
            transfers_out = []
            for p in players:
                if (squad[p, gw].value() or 0) > 0.5:
                    player_info = player_data.get((p, gw), {})
                    is_captain = 1 if (captain[p, gw].value() or 0) > 0.5 else 0
                    is_lineup = 1 if (lineup[p, gw].value() or 0) > 0.5 else 0
                    is_vice = 1 if (vicecap[p, gw].value() or 0) > 0.5 else 0
                    picks.append([
                        p,
                        player_info.get('position', ''),
                        player_info.get('team', ''),
                        player_info.get('value', 0),
                        player_info.get('xP', 0),
                        is_lineup,
                        is_captain,
                        is_vice
                    ])
                if gw > start_gameweek:
                    if (transfer_in[p, gw].value() or 0) > 0.5:
                        transfers_in.append(p)
                    if (transfer_out[p, gw].value() or 0) > 0.5:
                        transfers_out.append(p)

            picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
            picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
            
            results.append({
                'gameweek': gw,
                'picks': picks_df,
                'transfers_in': transfers_in,
                'transfers_out': transfers_out,
                'free_transfers': free_transfers[gw].value(),
                'paid_transfers': paid_transfers[gw].value()
            })

        total_xp = pulp.value(model.objective)
        print(f'Total expected points for all gameweeks: {total_xp}')

        return {'results': results, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None

#### Run optimiser

In [29]:
team_id = 41155

budget = 102.4
start_gameweek = 32
end_gameweek = 37
bench_boost = 33
free_hit = 34
wildcard = 0

In [30]:
projections_df = scrape_projections()

base_team = fpl_team(team_id, start_gameweek - 1)  # returns a list of player names

Extracted projections table


In [38]:
projections_df.tail()

,name,team,position,value,gameweek,xP
3619,Mosquera,WOL,DEF,3.8,36,0.0
3620,Pond,WOL,DEF,3.9,36,0.0
3621,Edozie,WOL,MID,4.5,36,0.0
3622,Okoduwa,WOL,DEF,4.0,36,0.0
3623,Mane,WOL,FWD,4.5,36,0.0


In [ ]:
result = solve_multi_period_fpl(budget, projections_df, start_gameweek, end_gameweek, 
                                bench_boost_gameweek = bench_boost, free_hit_gameweek = free_hit, wildcard_gameweek=wildcard,
                                base_team = base_team)

if result:
    for gw_result in result['results']:
        print(f"Gameweek {gw_result['gameweek']}:")
        print(gw_result['picks'])
        if gw_result['gameweek'] > 1:
            print("Transfers In:", gw_result['transfers_in'])
            print("Transfers Out:", gw_result['transfers_out'])
        print(f"Free Transfers: {gw_result['free_transfers']}")
        print(f"Paid Transfers: {gw_result['paid_transfers']}")
        print("\n")

    print(f"Total Expected Points: {result['total_xp']}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/a576ffd7f6f145b29b60b2115484771c-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/a576ffd7f6f145b29b60b2115484771c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9257 COLUMNS
At line 71834 RHS
At line 81087 BOUNDS
At line 90411 ENDATA
Problem MODEL has 9252 rows, 9323 columns and 38894 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 426.344 - 0.04 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 5238 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 96 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 96 strengthened rows, 0 su

In [ ]:
if result:
    markdown_output = ""

    # Add picks for each gameweek to Markdown
    for gw_result in result['results']:
        markdown_output += f"### Gameweek {gw_result['gameweek']} Picks\n\n"
        markdown_output += gw_result['picks'].to_markdown(index=False)
        markdown_output += "\n\n"

    # Add transfers summary to Markdown
    markdown_output += "### Transfers Summary\n\n"

    transfer_records = []
    for gw_result in result['results'][1:]:
        transfers_in = gw_result['transfers_in']
        transfers_out = gw_result['transfers_out']
        max_transfers = max(len(transfers_in), len(transfers_out))

        for i in range(max_transfers):
            transfer_in = transfers_in[i] if i < len(transfers_in) else ''
            transfer_out = transfers_out[i] if i < len(transfers_out) else ''
            transfer_records.append({'Gameweek': gw_result['gameweek'], 'Transfers In': transfer_in, 'Transfers Out': transfer_out})

    # Create a single DataFrame for all transfers
    transfers_df = pd.DataFrame(transfer_records)

    # Convert transfers DataFrame to Markdown
    markdown_output += transfers_df.to_markdown(index=False)

    # Print the Markdown output
    print(markdown_output)
else:
    print("Optimization did not find an optimal solution.")

In [24]:
df = pd.read_csv('../data/fpl_predictions.csv')
df = df.melt(id_vars=["name", "team", "position", "value"], 
                    value_vars=["32", "33", "34", "35", "36", "37"],
                    var_name="gameweek", 
                    value_name="xP")
df['season'] = "2024-25"
cols_opt = [
    "name",
    "position",
    "team",
    "xP",
    "value",
    "gameweek",
    "season"
    ]

df_optimisation = df[cols_opt].sort_values(by=['gameweek', 'season'], ascending=[True, True])
df_optimisation['gameweek'] = df_optimisation['gameweek'].astype('int')
df_optimisation['value'] = df_optimisation['value'].astype('float64')
df_optimisation['xP'] = df_optimisation['xP'].astype('float64')
print(df_optimisation)

             name position               team     xP  value  gameweek   season
0         M.Salah      MID          Liverpool   8.25   13.8        32  2024-25
1          Palmer      MID            Chelsea   8.17   10.7        32  2024-25
2            Saka      MID            Arsenal   6.13   10.3        32  2024-25
3            Isak      FWD          Newcastle  10.36    9.5        32  2024-25
4     B.Fernandes      MID            Man Utd   4.18    8.6        32  2024-25
...           ...      ...                ...    ...    ...       ...      ...
1759        Lewis      DEF           Man City   0.98    4.3        37  2024-25
1760   Jota Silva      MID  Nottingham Forest   1.00    5.9        37  2024-25
1761      Odobert      MID          Tottenham   1.07    5.3        37  2024-25
1762    Cresswell      DEF           West Ham   1.41    3.9        37  2024-25
1763        Winks      MID          Leicester   1.21    4.4        37  2024-25

[1764 rows x 7 columns]
